In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [5]:
df = pd.read_csv('Train_data.csv')

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV, validation_curve
from sklearn.preprocessing import StandardScaler, QuantileTransformer, MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

In [7]:
col_transform = [col for col in df.columns if col not in ['Target']]

In [13]:
X = df[col_transform]
y = df['Target']

In [15]:
transformer = ColumnTransformer(transformers = [('scaling',RobustScaler(),
                                                 col_transform)])

In [16]:
from xgboost import XGBClassifier

In [17]:
xgb = XGBClassifier()

In [18]:
pipeline_xgb = Pipeline(steps = [('transformer',transformer),('xgb',XGBClassifier())])

In [19]:
cv_score_xgb = cross_val_score(pipeline_xgb,X,y,cv =5)

In [20]:
cv_score_xgb.mean()

0.716635129576306

In [21]:
params_xgb = {'xgb__n_estimators':[100, 300, 500, 750, 800, 1200] ,
              'xgb__max_depth': [5, 10, 15, 20, 25,30],'xgb__learning_rate':[0.01, 0.1, 1, 0.001, 0.5]}

In [22]:
grid_xgb = GridSearchCV(pipeline_xgb,param_grid=params_xgb,cv=5,verbose=1,n_jobs=4)

In [23]:
grid_xgb.fit(X,y)

Fitting 5 folds for each of 180 candidates, totalling 900 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('transformer',
                                        ColumnTransformer(transformers=[('scaling',
                                                                         RobustScaler(),
                                                                         ['GP',
                                                                          'MIN',
                                                                          'PTS',
                                                                          'FGM',
                                                                          'FGA',
                                                                          'FG%',
                                                                          '3P '
                                                                          'Made',
                                                                          '3PA',
                         

In [24]:
grid_xgb.best_params_

{'xgb__learning_rate': 0.5, 'xgb__max_depth': 10, 'xgb__n_estimators': 800}

In [25]:
pipeline_xgb_best = Pipeline(steps = [('transformer',transformer),('xgb',XGBClassifier(n_estimators = 800,max_depth = 10, learning_rate = 0.5))])

In [26]:
cv_score_xgb_best = cross_val_score(pipeline_xgb_best,X,y,cv =5)

In [27]:
cv_score_xgb_best.mean()

0.7338667215137804

In [29]:
X_train = X
y_train = y

In [30]:
pipeline_xgb_best.fit(X_train,y_train)

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('scaling', RobustScaler(),
                                                  ['GP', 'MIN', 'PTS', 'FGM',
                                                   'FGA', 'FG%', '3P Made',
                                                   '3PA', '3P%', 'FTM', 'FTA',
                                                   'FT%', 'OREB', 'DREB', 'REB',
                                                   'AST', 'STL', 'BLK',
                                                   'TOV'])])),
                ('xgb',
                 XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, early_stopp...
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                               learning_ra

In [32]:
X_test = pd.read_csv('Test_data.csv')

In [33]:
predictions_xgb = pipeline_xgb_best.predict(X_test)

In [34]:
predictions = pd.DataFrame(predictions_xgb, columns = ['prediction'])
predictions.to_csv('submission_5.csv')